# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load the CSV file
csv_file = "../output_data/cities.csv"
city_data = pd.read_csv(csv_file)
city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saskylakh,71.9167,114.0833,33.75,87,94,10.11,RU,1619674013
1,1,loukhi,66.0764,33.0381,29.75,72,100,3.98,RU,1619674014
2,2,mataura,-46.1927,168.8643,55.99,94,96,14.00,NZ,1619674014
3,3,butaritari,3.0707,172.7902,82.85,73,94,10.00,KI,1619674015
4,4,ushuaia,-54.8000,-68.3000,35.60,80,40,16.11,AR,1619674015
...,...,...,...,...,...,...,...,...,...,...
579,579,borovoy,55.4333,86.0833,51.80,58,0,8.95,RU,1619674224
580,580,lufilufi,-13.8745,-171.5986,82.40,89,75,10.36,WS,1619674224
581,581,collie,-33.3667,116.1500,64.99,83,97,13.00,AU,1619674225
582,582,chumphon,10.3333,98.8333,91.40,62,40,5.75,TH,1619674225


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure GMaps
gmaps.configure(api_key=g_key)

# Set the data set
locations = city_data[["Lat", "Lng"]].astype(float)
humidity  = city_data["Humidity"].astype(float)

In [6]:
# Configure the map layout
fig = gmaps.figure(layout={
        'width': '800px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
}, zoom_level=2, center=(46.0, -5.0))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Search Criterias
filtered_city_data = city_data.loc[(city_data['Max Temp']<80) & 
                                   (city_data['Max Temp']>70) & 
                                   (city_data['Wind Speed']<10) & 
                                   (city_data['Cloudiness']==0),:]
filtered_city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
105,105,abdanan,32.9926,47.4198,70.52,28,0,6.73,IR,1619674052
185,185,matay,28.4190,30.7792,74.53,27,0,8.90,EG,1619674081
186,186,shache,38.4167,77.2406,79.12,8,0,1.63,CN,1619674081
278,278,beloha,-25.1667,45.0500,76.86,58,0,5.50,MG,1619674114
375,375,aksu,41.1231,80.2644,78.80,9,0,1.36,CN,1619674148
382,382,korla,41.7597,86.1469,77.22,13,0,7.76,CN,1619674151
414,414,empalme,27.9667,-110.8167,71.01,41,0,5.01,MX,1619674163
437,437,gasa,31.5000,34.4667,72.00,45,0,3.18,PS,1619674171
505,505,yulara,-25.2406,130.9889,77.00,18,0,9.22,AU,1619674195
561,561,gat,31.6100,34.7642,72.00,45,0,3.11,IL,1619674217


In [8]:
len(filtered_city_data)

11

In [9]:
# NaN values dropped if present
filtered_city_data = filtered_city_data.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = []

#Loop to get the hotels using google maps
for index, row in filtered_city_data.iterrows():
    lat = filtered_city_data['Lat'][index]
    lng = filtered_city_data['Lng'][index]
    params = {
        "location": f"{lat},{lng}",
        "keyword": "hotel",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }
    try:
        response = requests.get(base_url, params=params).json()
        
        hotels.append(response['results'][0]['name'])    
    except IndexError:
        hotels.append("")
        pass

In [11]:
# New Data Frame with the first Hotel found
hotel_df = filtered_city_data
hotel_df['Hotel Name'] = hotels
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
105,105,abdanan,32.9926,47.4198,70.52,28,0,6.73,IR,1619674052,Kabirkooh Star Hotel
185,185,matay,28.4190,30.7792,74.53,27,0,8.90,EG,1619674081,
186,186,shache,38.4167,77.2406,79.12,8,0,1.63,CN,1619674081,Xinsheng Hotel
278,278,beloha,-25.1667,45.0500,76.86,58,0,5.50,MG,1619674114,
375,375,aksu,41.1231,80.2644,78.80,9,0,1.36,CN,1619674148,Jianshe Hotel
382,382,korla,41.7597,86.1469,77.22,13,0,7.76,CN,1619674151,The Garden Hotel Korla
414,414,empalme,27.9667,-110.8167,71.01,41,0,5.01,MX,1619674163,Motel El Nido
437,437,gasa,31.5000,34.4667,72.00,45,0,3.18,PS,1619674171,
505,505,yulara,-25.2406,130.9889,77.00,18,0,9.22,AU,1619674195,Sails in the Desert
561,561,gat,31.6100,34.7642,72.00,45,0,3.11,IL,1619674217,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))